In [80]:
!pip install -q snscrape 

In [81]:
import os 
import pandas as pd 
from os import listdir
from datetime import date, datetime, timedelta

In [82]:
def date_range(start, end):
    delta = end - start  # as timedelta
    days = [start + timedelta(days=i) for i in range(delta.days + 1)]
    return_days = [day.date() for day in days]
    return return_days

start_date = datetime(2022, 3, 12)
end_date = datetime(2022, 4, 24)
    
date_range_for_data = date_range(start_date, end_date)


def create_tuples(date_range_list): 
    return_list = []
    
    # example [(2021-10-31, 2021-11-1), (2021-11-1, 2021-11-2)]
    for i in range(len(date_range_list)): 
        if not(date_range_list[i] == date_range_list[len(date_range_list)-1]):
            return_list.append((date_range_list[i], date_range_list[i+1]))

    return return_list

date_range_tuple = create_tuples(date_range_for_data)

In [83]:
# Get tweets function with 
# inputs (max_results, search_term, from_date, end_date)
# For us the from and end date will be the same 
# it will output a csv in the current directory with date as the start of the file
# and the search term (query at the start as well) 
def get_tweets(tweet_count, text_query, since_date, until_date):
    # Using OS library to call CLI commands in Python
    os.system('snscrape --jsonl --max-results {} --since {} twitter-search "{} until:{}"> text-query-tweets.json'.format( tweet_count, since_date, text_query, until_date))

    # Reads the json generated from the CLI command above and creates a pandas dataframe
    tweets_df = pd.read_json('text-query-tweets.json', lines=True)
    tweets_df.to_csv(f'{since_date}_{text_query}_tweets.csv', sep=',', index=False)

# Do not run this anymore

In [6]:
tweet_count = 1000
text_query = "russia ukraine oil"

for date in date_range_tuple:
    get_tweets(tweet_count, text_query, date[0], date[1])

# Data Preprocessing 

#### Run this before you proceed with cleaning up and filtering the data 

#### final_dict - Is this dictionary that contains all the data according to dates as the key 

In [35]:
current_path = os.getcwd()

path_tweets = f'{current_path}/russia_ukraine_tweet/'

files = listdir(path_tweets)

#read files which end with csv 
# put them in a list 
#Input: all files in the directory list of files 
#Output: filter only the files 
def filter_csv(files):
    
    return_list = []
    
    for f in files: 
        if f.endswith('csv'):
            return_list.append(f)
    
    return return_list 

# Tested - Works well
files_list = filter_csv(files)
            
    
# write another function which makes a dictionary from a list of csv files 
# Prerequisite for the dictionary - filenames need to begin with dates 
# Key: Date Value: CSV 
# Input list of csv files
# Output : Dictionary - date and its constituent df 
def create_data_dictionary(files_list): 
    return_dict = {}
    files_with_no_data = [] 
    current_path = os.getcwd()
    data_path = f'{current_path}/russia_ukraine_tweet'
    for f in files_list:
        try:
            value = pd.read_csv(f'{data_path}/{f}')
            key = f[0:10]
            return_dict[key] = value
        except:
            files_with_no_data.append(f)
            continue 
            
    # to check if there are any files which are empty 
    # there is another way to filter and add it in this function itself but I have broken down 
    # for easier understanding. 
    return ((return_dict, files_with_no_data))

In [45]:
final_dict, files_with_no_data = create_data_dictionary(files_list)

# without days with no data 
final_days_of_data = len(final_dict.keys())

# input : dictionary , list of files 
# out : dictionary updated with file names but value is a string - 'no data'
def insert_null_val(dict_, files): 
    for i in files: 
        dict_[i[0:10]] = 'no value'
    
    return dict_

#with days with no data 
final_dict = insert_null_val(final_dict, files_with_no_data)

166


## Cleaning up each dataframe individually 

In [79]:
# TODO: 
# Take a sample 
# Filter the data 
# Create functions and test on sample df shown below
# source = [twitter for android, twitter for ios, twitter web app, news sources] 
# language = en (english tweets only)
# clean up tweets 



sample_df = final_dict['2022-03-14']